In [ ]:
import os
from dotenv import load_dotenv
from datetime import datetime, timedelta, timezone

# Required libs to measure / track the time to do things
import time # for some performance testing
from tqdm.autonotebook import tqdm

# Required libs to authenticate via OAuth Client Credentials and fetch the data from CloudIQ REST API
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

# Required libs to process the data
import pandas as pd
import numpy as np

# Required libs to visualize
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# CloudIQ REST API endpoints
CIQ_TOKEN_URL = 'https://cloudiq.apis.dell.com/auth/oauth/v2/token'
CIQ_BASE_API_URL = 'https://cloudiq.apis.dell.com/cloudiq/rest/v1/'

In [ ]:
# You don't want your API credentials to leak, do you?
# just create a ".env" file in the same directory as the Jupyter notebooks, with those 2 variables defined:
# CLIENT_ID=<your_client_id>
# CLIENT_SECRET=<your_client_secret>
load_dotenv()

client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

print('OAuth2 Client Credentials ID used:', client_id)

In [ ]:
# Authentication to CloudIQ REST API
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)
token = oauth.fetch_token(token_url=CIQ_TOKEN_URL, client_id=client_id, client_secret=client_secret)

print('OAuth2 Token type:', token['token_type'], '- Expires in:', token['expires_in'])

In [ ]:
# Fetching all servers
# A simple REST API call to CloudIQ REST API, adding the resource name to the prefix, and passing some parameters to filter instances, and to select attributes
params = { 
#     'filter': "country eq 'US'",
    'select': 'system_id,country,state,city,site_name,system,version,model,system_tags,power_state,power_consumption,health_score,inlet_temperature,cpu_usage_percent,memory_usage_percent'
}

r = oauth.get(CIQ_BASE_API_URL + 'server_systems', params=params)
print('Status Code:', r.status_code)

In [ ]:
# How many instances / servers do we have?
print('Number of instances:', r.json()['paging']['total_instances'])
# r.json()

In [ ]:
# Creating DataFrame
df_object = pd.json_normalize(r.json(), record_path =['results'])

print('Dataframe size:', df_object.shape)
# df_object

In [ ]:
# Filtering systems with no data
df_object[~df_object['cpu_usage_percent'].isnull() & \
          ~df_object['inlet_temperature'].isnull() & \
          ~df_object['memory_usage_percent'].isnull() & \
          ~df_object['power_consumption'].isnull()].shape

In [ ]:
# Fetching some metrics - functions
MAX_ID = 5 # Not all resource ids at the same time, but in chunks

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
# Transforming the metrics data into a DataFrame
def flatten(s):
    metrics = s['metrics']
    results = s['results']
    
    # Creating a record per object, per metric and per timestamp to then create a DataFrame
    data = [{'object': metric['id'], 'datetime': ts['timestamp']} | dict(zip(metrics, ts['values']))
            for metric in results
            for ts in metric['timestamps']]
    
    df = pd.DataFrame.from_records(data)
    if not df.empty:
        df['datetime'] = pd.to_datetime(df['datetime'])

    return df

# Crafting the POST query
def post_metric_json(from_dt, to_dt, ids, resource, metrics, interval):
    return {
        'from': from_dt.isoformat(),
        'to': to_dt.isoformat(),
        'resource_type': resource,
        'ids': ids,
        'interval': interval,
        'metrics': metrics
    }
    
# Fetching timeseries (values over time) / metrics between 2 dates
def get_metrics_df(from_dt, to_dt, ids, resource, metrics, interval):
    # Start with an empty dataframe
    data = pd.DataFrame()

    with tqdm(total=len(ids)) as pbar:
        # Split the request into sub-requests (resource ids)
        for ids_subset in chunks(ids, MAX_ID):
            # Set the from and to, maybe we can fetch all data in 1 call / response
            from_dt_post = from_dt
            to_dt_post = to_dt
            
            # Send requests until we have all content - i.e. until no more HTTP 206 / Partial Content
            nb_request = 0
            while True:
                json_to_post = post_metric_json(from_dt_post, to_dt_post, ids_subset, resource, metrics, interval)
                nb_request += 1

                # Post the query to fetch the metrics
                r = oauth.post(CIQ_BASE_API_URL + 'metrics/query', json=json_to_post)
                # Append response to dataframe
                new_data = flatten(r.json())
                data = pd.concat([data, new_data])
                # If no more data, stop here
                if r.status_code == 200:
                    break
                # If Partial content, continue by shifting the from date to the one received in the response
                elif r.status_code == 206:
                    if not new_data.empty:
                        from_dt_post = max(new_data['datetime'])
                    # Stop if the response returns no data
                    else:
                        break

            # print('Number of API calls:', nb_request)
            # print('Updated dataframe size:', data.shape)
            pbar.update(len(ids_subset))
        
    return data

In [ ]:
# Fetching some metrics over time
# between 25 days ago and now
TO = datetime.now(timezone.utc).replace(microsecond=0)
FROM = TO - timedelta(days=25)
print('From:', FROM)
print('To:', TO)

RESOURCE = 'server_system'

METRICS = [
        'inlet_temperature',
        'power_consumption',
        'system_energy',
        'cpu_usage_percent'
]

INTERVAL = 'PT15M'

# for all servers which have a temperature at the system's level
SERVERS = df_object[~df_object['inlet_temperature'].isnull()]['id'].tolist()

start = time.time()
df_all_metrics_data = get_metrics_df(FROM, TO, SERVERS, RESOURCE, METRICS, INTERVAL)
end = time.time()

print('Dataframe size:', df_all_metrics_data.shape)
print('Time (ms):', (end-start)*1000)
# df_all_metrics_data

In [ ]:
# Merge metrics with metadata / system level attributes
df_all_metrics_data = pd.merge(df_all_metrics_data, df_object[['system', 'country', 'city', 'site_name']], how='left', left_on='object', right_on='system')
df_all_metrics_data

In [ ]:
# Treemap of the temperature per system
fig = px.treemap(df_object, path=[px.Constant("ALL"), 'country', 'city', 'site_name', 'system'], values='inlet_temperature',
                    color='inlet_temperature', hover_data=['id', 'city'],
                    custom_data=df_object[['country', 'city', 'site_name']],
                    color_continuous_scale='Jet')
fig.update_traces(root_color='lightgrey', hovertemplate='Server: %{label}<br>Temperature: %{color}<extra></extra><br><br>Site: %{customdata[0]} / %{customdata[1]}')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [ ]:
# Heatmap of the temperature - ToDo: per group...
fig = go.Figure(data=go.Heatmap(
        hovertemplate='Server: %{y}<br>Date: %{x}<br>Temperature: %{z}<extra></extra><br><br>Site: %{customdata}',
        z=df_all_metrics_data['inlet_temperature'],
        x=df_all_metrics_data['datetime'],
        y=df_all_metrics_data['object'],
        customdata=df_all_metrics_data['city'],
        colorscale='RdBu_r')) # RdBu_r OR Jet

fig.update_layout(
    title='Temperature over time',
    xaxis_nticks=24,
    height=800)

fig.show()

In [ ]:
# Line chart with 2 metrics (Power, CPU)
# Create figure with 2 Y axis
# Specify the filter to get the specific server you want to focus on:
object_filter = df_all_metrics_data['object'] == '<a_server>'

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=df_all_metrics_data[object_filter]['datetime'], y=df_all_metrics_data[object_filter]['power_consumption'], name='Power (W)', yaxis='y')
)

fig.add_trace(
    go.Scatter(x=df_all_metrics_data[object_filter]['datetime'], y=df_all_metrics_data[object_filter]['cpu_usage_percent'], name='CPU Utilization (%)', yaxis='y2')
)

fig.update_layout(
    title_text="CPU Utilization vs Power consumption"
)

fig.update_xaxes(title_text="Date and time")
fig.update_xaxes(rangeslider_visible=True)

fig.update_yaxes(title_text="<b>Power (W)</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>CPU Utilization (%)</b>", secondary_y=True)

fig.show()